In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras



print(tf.__version__)
print(sys.version_info)

for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)
matplotlib 3.2.2
numpy 1.18.5
pandas 1.0.5
sklearn 0.21.2
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


# 常量

In [2]:
# 常量
t = tf.constant([[1.,2.,3.],[4.,5.,6.]])

# index操作
print(t)
print(t[:,1:])

# 只拿出第二列的值
print(t[...,1])

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)


In [4]:
# ops

# 加法
print(t+10)

# 平方
print(tf.square(t))

# 乘以自己的转置 (2,3)*(3,2)=(2,2)
print(t @ tf.transpose(t))

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [8]:
# 与numpy转换

print(t.numpy())

print(np.square(t))

np_t = np.array([[1.,2.,3.],[4.,5.,6.]])

print(tf.constant(np_t))

[[1. 2. 3.]
 [4. 5. 6.]]
[[ 1.  4.  9.]
 [16. 25. 36.]]
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float64)


In [9]:
# 处理一维的值
# Scalars

t= tf.constant(2.784)
print(t.numpy())
print(t.shape)

2.784
()


In [12]:
# 处理字符串 strings
t=tf.constant("cafe")
print(t)

print(tf.strings.length(t))

print(tf.strings.length(t,unit="UTF8_CHAR"))

print(tf.strings.unicode_decode(t,"UTF8"))

tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)


In [13]:
#  处理字符串数组
t= tf.constant(["cafe","coffee","咖啡"])
print(tf.strings.length(t,unit="UTF8_CHAR"))
r = tf.strings.unicode_decode(t,"UTF8")
print(r)

# RaggedTensor：不完整n维矩阵，每行长度一样，第一个长度是4.第二个长度是6，第三个长度是2

tf.Tensor([4 6 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


In [17]:
# RaggedTensor
r = tf.ragged.constant([[1,2],[3,4,5],[],[8]])

print(r)
print(r[1])
print(r[1:2])

<tf.RaggedTensor [[1, 2], [3, 4, 5], [], [8]]>
tf.Tensor([3 4 5], shape=(3,), dtype=int32)
<tf.RaggedTensor [[3, 4, 5]]>


In [25]:
# ragged tensor 的操作
r2 = tf.ragged.constant([[9,10,2],[],[45,67]])

# axis=0 在行的维度上进行拼接，4行+3行拼接后就变成7行
print(tf.concat([r,r2],axis=0))

<tf.RaggedTensor [[1, 2], [3, 4, 5], [], [8], [9, 10, 2], [], [45, 67]]>


In [26]:
# axis=1 在列的维度上进行拼接，要求必须要有相同的行数
print(tf.concat([r,r2],axis=1))

InvalidArgumentError: Input tensors have incompatible shapes.
Condition x == y did not hold.
First 1 elements of x:
[3]
First 1 elements of y:
[4]


In [24]:
# 定义一个想听行数的raggedTensor
r3 = tf.ragged.constant([[],[22,55],[32,45,234],[767,7674,53]])
# 就可以在列的维度上进行拼接
print(tf.concat([r,r3],axis=1))

<tf.RaggedTensor [[1, 2], [3, 4, 5, 22, 55], [32, 45, 234], [8, 767, 7674, 53]]>


In [29]:
# ragged tensor 可以变成普通的tensor
# 缺失的值都用0在正常值的后面进行填充
print(r.to_tensor())

tf.Tensor(
[[1 2 0]
 [3 4 5]
 [0 0 0]
 [8 0 0]], shape=(4, 3), dtype=int32)


In [30]:
# sparse tensor 稀疏矩阵
s = tf.SparseTensor(indices=[[0,1],[1,0],[2,3]],values=[1.,2.,3.],dense_shape=[3,4])
print(s)

# 转换为密集矩阵
print(tf.sparse.to_dense(s))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [32]:
# ops on sparse tensors 

# 乘法
s2 = s *2.0
print(s2)

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([2. 4. 6.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


TypeError: unsupported operand type(s) for +: 'SparseTensor' and 'int'

In [33]:
# 加法 报错
s+1

TypeError: unsupported operand type(s) for +: 'SparseTensor' and 'int'

In [34]:
# 和一个密集矩阵相乘得到一个密集矩阵
s4 =tf.constant([[10.,20.],[23.,43.],[323.,3535.],[454.,343.]])
tf.sparse.sparse_dense_matmul(s,s4)

<tf.Tensor: id=484, shape=(3, 2), dtype=float32, numpy=
array([[  23.,   43.],
       [  20.,   40.],
       [1362., 1029.]], dtype=float32)>

In [36]:
# sparse tensor 稀疏矩阵 如果把索引的顺序打乱就会报错，需要调用tf.spqarse.reorder()
s5 = tf.SparseTensor(indices=[[1,0],[0,1],[2,3]],values=[1.,2.,3.],dense_shape=[3,4])
print(s5)

s6 = tf.sparse.reorder(s5)
# 转换为密集矩阵
print(tf.sparse.to_dense(s6))

SparseTensor(indices=tf.Tensor(
[[1 0]
 [0 1]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 2. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


# 变量Variables

In [41]:
# Variables

v = tf.Variable([[1,2,4],[3,5,6]])

print(v)

print(v.value())
print(v.numpy())

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 4],
       [3, 5, 6]])>
tf.Tensor(
[[1 2 4]
 [3 5 6]], shape=(2, 3), dtype=int32)
[[1 2 4]
 [3 5 6]]


In [42]:
# assign value重新赋值
v.assign(2*v)
print(v.numpy())

v[0,1].assign(42)
print(v.numpy())

v[1].assign([7,8,9])
print(v.numpy())

[[ 2  4  8]
 [ 6 10 12]]
[[ 2 42  8]
 [ 6 10 12]]
[[ 2 42  8]
 [ 7  8  9]]


In [43]:
# 只能用assign进行赋值不能用=
v[1]=[7,8,9]

TypeError: 'ResourceVariable' object does not support item assignment